# Task 1 #


А теперь давай усложним задачу.Ты знаешь, что такое подзапрос? Это как запрос, но только внутри запроса. Чем он может быть полезен? Например, с его помощью можно сделать определенную агрегацию для выбранных данных. Однако следует помнить, что в первую очередь выполняются вложенные запросы, и только после этого — основной запрос.

Что тебе нужно сделать:

In [1]:
import sqlite3
import pandas as pd


## Создай соединение с базой данных с помощью библиотеки sqlite3. ##
----------

In [2]:
# создание подключения к базе данных
conn = sqlite3.connect('../data/checking-logs.sqlite')

## Получи схему таблицы pageviews с помощью pd.io.sql.read_sql и запроса PRAGMA table_info(pageviews); ##
----------

In [3]:
# Получаем схему таблицы pageviews
schema = pd.io.sql.read_sql("PRAGMA table_info(pageviews);", conn)
print(schema)

   cid      name       type  notnull dflt_value  pk
0    0     index    INTEGER        0       None   0
1    1       uid       TEXT        0       None   0
2    2  datetime  TIMESTAMP        0       None   0


## Получи только первые 10 строк таблицы pageviews, чтобы проверить, как она выглядит. ##
------

In [4]:
# Получаем только первые 10 строк таблицы pageviews
pageviews = pd.io.sql.read_sql("SELECT * FROM pageviews LIMIT 10;", conn)
print(pageviews)

   index      uid                    datetime
0      0  admin_1  2020-04-17 12:01:08.463179
1      1  admin_1  2020-04-17 12:01:23.743946
2      2  admin_3  2020-04-17 12:17:39.287778
3      3  admin_3  2020-04-17 12:17:40.001768
4      4  admin_1  2020-04-17 12:27:30.646665
5      5  admin_1  2020-04-17 12:35:44.884757
6      6  admin_1  2020-04-17 12:35:52.735016
7      7  admin_3  2020-04-17 12:36:21.401412
8      8  admin_3  2020-04-17 12:36:22.023355
9      9  admin_1  2020-04-17 13:55:19.129243


## Получи подтаблицу с помощью только одного запроса ##
--------

* используются только uid и datetime;
* используются только данные пользователей (user_*), и не используются данные администраторов;
* сортировка выполняется по uid в порядке возрастания;
* столбец индекса — это datetime;
* datetime преобразован в DatetimeIndex;
* имя датафрейма — pageviews.

In [5]:
# Получаем подтаблицу с нужными полями и условиями
pageviews = pd.io.sql.read_sql("SELECT uid, datetime FROM pageviews WHERE uid LIKE 'user_%' AND uid NOT LIKE 'admin_%' ORDER BY uid ASC;", 
                               conn, index_col='datetime', parse_dates=['datetime'])
print(pageviews)

                                uid
datetime                           
2020-04-26 21:53:59.624136   user_1
2020-04-26 22:06:19.478143   user_1
2020-04-26 22:12:09.614497   user_1
2020-04-30 19:29:01.831635   user_1
2020-05-05 20:26:32.894852   user_1
...                             ...
2020-04-29 16:51:21.877630  user_30
2020-05-09 20:30:47.034282  user_30
2020-05-22 11:30:18.368990   user_5
2020-05-21 16:28:28.217529   user_9
2020-05-21 16:36:40.915488   user_9

[987 rows x 1 columns]


## Для проверки

In [6]:
pageviews.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 987 entries, 2020-04-26 21:53:59.624136 to 2020-05-21 16:36:40.915488
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   uid     987 non-null    object
dtypes: object(1)
memory usage: 15.4+ KB


## Закрой соединение с базой данных. ##
------

In [7]:
conn.close()